In [1]:
import pandas as pd
import random
import numpy as np

from bs4 import BeautifulSoup
import requests
import re
import json
import csv

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

from time import sleep

pd.set_option('display.max_columns', None)


C:\Users\Isabel\AppData\Local\Temp\ipykernel_13752\3175742525.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open('../Fase1_datos/lista_comedia.json', 'r') as fichero:
    comedia = json.load(fichero)

In [4]:
len(comedia)

4588

In [7]:
busqueda = comedia[:500]
len(busqueda)

500

In [3]:
# variables donde iremos almacenando todo
info_por_peli = []
fallos = []

In [8]:
# llegar a web:
driver = webdriver.Chrome()
driver.get('https://www.imdb.com/')
driver.maximize_window()
sleep(random.randint(1,3))


# rechazar cookies
driver.find_element('css selector', '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
sleep(random.randint(1,3))


# entrando en cada título
for peli in busqueda:
    lista_info = [f'{peli[0]}',]

    try:
    # metemos el ID en el buscador
        try:
            driver.find_element('css selector', '#suggestion-search').send_keys(peli[0], Keys.ENTER)
            sleep(random.randint(1,3))

            # extraemos info
            info = driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text
            lista_info.append(info.split('\n'))

            info_por_peli.append(lista_info)
        except:
            driver.find_element('css selector', '#error > div.error_attrib > a').click()
            
    except:
        fallos.append(peli)

In [9]:
len(info_por_peli)

498

In [12]:
lista_tuplas = []

In [13]:
for peli in info_por_peli:
    
    patron = re.compile(r'([a-z])([A-Z])')
    patron_duracion = re.compile(r'\b(?:\d+h\s*)?\d+min\b')
    tupla_datos = [peli[0],] 

    # listas intermedias para ir checkeando
    direccion = []
    guion = []
    nota = []
    duracion = []
    argumento = []
        
    for indice, linea in enumerate(peli[1]):
               
        
        # directores    
        if 'Dirección' in linea:
                                
            directores = patron.sub(r'\1, \2', peli[1][indice+1])
            direccion.append(directores)            


        # guionistas
        elif 'Guión' in linea:
            
            guionistas = patron.sub(r'\1, \2', peli[1][indice+1])
            guion.append(guionistas)
                
        
        # nota y duracion
        elif 'PUNTUACIÓN EN IMDb' in linea:
            nota.append(float(peli[1][indice+1].replace(',', '.')))
            
            if r'\d+h' or r'\d+min' in peli[1][indice-1]:  
                duracion.append(peli[1][indice-1])
            
                    
        # argumento
        elif 'Añade un argumento en tu idioma' in linea:
            argumento.append(peli[1][indice+1])

    # añadimos a la tupla los datos de la lista intermedia
    for elemento in [direccion, guion, nota, duracion, argumento]:
        if elemento != []:
            tupla_datos.append(elemento[0])
        else:
            tupla_datos.append(np.nan)
    
    lista_tuplas.append(tupla_datos)

In [15]:
lista_tuplas

[['tt0119495',
  'Adrian Fulle',
  'Adrian Fulle, David J. Miller, Daniel Bingley',
  5.0,
  '1h 26min',
  'A group of raucous college buds living the Dorm life, decide to have a blowout while most of their friends are away for the Thanksgiving holiday. With LOVE 101 being the toughest course this semester, will they pass the final exam?'],
 ['tt0119806',
  'David Elliot, Mark Huppin',
  'David Elliot, Mark Huppin',
  6.0,
  '1h 38min',
  "Three longtime pals live in San Francisco: Darin is married to Natalie, but their family life is too still and Darin is not happy about it, Matt is ladies' magnet but tired of it, trying to get something stable from his relationships, and goofy Kevin doesn't have any success with women at all. Darin starts an affair with his friend's wife Maxine and Matt falls deeply ... Leer todo"],
 ['tt0120202',
  'David Mamet',
  'David Mamet',
  6.7,
  '1h 45min',
  'A movie crew invades a small town whose residents are all too ready to give up their values for 

In [11]:
with open('ComediaWIP.json', 'w') as fichero:
    json.dump(lista_tuplas, fichero)

In [10]:
len(fallos)

0